
# HW6 Diffusion Model

**Sources:**
- Github implementation [Denoising Diffusion Pytorch](https://github.com/lucidrains/denoising-diffusion-pytorch)
- Papers on Diffusion models ([Dhariwal, Nichol, 2021], [Ho et al., 2020] ect.)


## Import Packages and Set Seeds

In [1]:
!pip install einops
!pip install transformers
!pip install ema_pytorch
!pip install accelerate
!pip install stylegan2_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 63.8 MB/s eta 0:00:00:00:010:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 49.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 57.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 9.7 MB/s eta 0:00:00
  Created wheel for vector-quantize-pytorch: filename=vector_quantize_pytorch-0.1.0-py3-none-any.whl size=2573 sha256=3b5297ae0ecd2d93cfae08509755bdaf8aa507b5641a1790cc18998c5dac46d2
  Stored in directory: /root/.cache/pip/wheels/dc/7d/46/de4b4836c2748c63414d7f8c1ed7b805d59ba8047f6fe2af

In [2]:
# import module
import os
import glob
import random
from datetime import datetime
from pathlib import Path
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms as T, utils
from torch import optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import numpy as np
import logging
from tqdm import tqdm


# seed setting
def same_seeds(seed):
    # Python built-in random module
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Torch
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(42)
workspace_dir = '/kaggle/input/diffusion'

Create dataset

In [3]:
class Dataset(Dataset):
    def __init__(self, folder, size):
        self.folder = folder
        self.paths = [p for p in Path(f'{folder}').glob(f'**/*.jpg')]
        self.transform = T.Compose([
            T.Resize((size, size)),
            T.ToTensor(),
        ])

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        path = self.paths[index]
        img = Image.open(path)
        return self.transform(img)

styleGan

In [4]:
# REF : https://github.com/lucidrains/stylegan2-pytorch

In [5]:
!stylegan2_pytorch --data /kaggle/input/diffusion/faces/faces \
--image-size 64 \
--num-train-steps 27000 \
--batch-size 16 \
--gradient-accumulate-every 1 \
--network-capacity 64 \
--aug-prob 0.35 \
--aug-types [color]

default</kaggle/input/diffusion/faces/faces>:   0%| | 50/27000 [01:05<9:46:45,  G: 6.05 | D: 0.43 | GP: 5.12
default</kaggle/input/diffusion/faces/faces>:   0%| | 100/27000 [02:02<9:10:34, G: -1.27 | D: 2.99 | GP: 3.40
default</kaggle/input/diffusion/faces/faces>:   1%| | 150/27000 [03:03<9:06:43, G: 2.57 | D: 0.75 | GP: 2.37
default</kaggle/input/diffusion/faces/faces>:   1%| | 188/27000 [03:46<8:58:48, ^C


# Inference

In [10]:
!stylegan2_pytorch --generate \
--trunc-psi 0.5 \
--load-from 27 \
--num_generate 1000 \
--num_image_tiles 1 \
--results_dir "./output1"

loading from version 1.8.9
100%|███████████████████████████████████████| 1000/1000 [00:28<00:00, 35.21it/s]
sample images generated at ./output1/default/generated-04-21-2023_14-20-53


In [11]:
os.mkdir("/kaggle/working/outputMR")

FileExistsError: [Errno 17] File exists: '/kaggle/working/outputMR'

In [12]:
temp_dataset1 = Dataset(os.path.join("/kaggle/working/output1", 'default'), 64)

ct = 0
for i in range(len(temp_dataset1)):
    str2 = str(temp_dataset1.paths[i])
    if(str2[-5]== 'r'):
        ct += 1
        torchvision.utils.save_image(temp_dataset1[i], os.path.join("./outputMR", f'{ct}.jpg'))
print(ct)

1000


In [13]:
%cd outputMR
!tar -zcf ../submissionMR.tgz *.jpg
%cd ..

/kaggle/working/outputMR
/kaggle/working
